In [ ]:
import time
import re
import sqlite3
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By 
import csv
import pandas as pd

# 예외 처리를 위한 모듈
from selenium.webdriver.support import expected_conditions as EC


# 드라이버 오픈
driver = webdriver.Chrome("C:/driver/chromedriver.exe")
#url='https://watcha.com/en-KR/users/RE952Z1gg5Q72'
url = 'https://watcha.com'
driver.get(url)

# 아이디, 비민번호 변수 지정
account = 'encore1@naver.com'
pw = 'encore'


driver.find_element_by_xpath('//*[@id="root"]/div/div/header/nav/div/div/ul/li[2]/button').click()

# 로그인 로직
#login.click()


driver.find_element_by_name('email').send_keys(account)

driver.find_element_by_name('password').send_keys(pw)
time.sleep(2)

# id, pw 입력 후 로그인 클릭
driver.find_element_by_xpath('//*[@id="root"]/div[2]/div/div/div/section/div/div/form/button').click()
time.sleep(3)

# 팝업끄기
driver.find_element_by_xpath('//*[@id="root"]/div[2]/div/div/header/div[1]/button').click()

time.sleep(2)

# 나의 도서페이지 가기
driver.get('https://watcha.com/en-KR/users/RE952Z1gg5Q72/contents/books')

# 더보기 누르기
driver.find_element_by_xpath('//*[@id="root"]/div/div/section/section/section/div[1]/div/header/div/div/a').click()


# 스크롤 제일 끝으로 내리기
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    #time.sleep(3)
    
time.sleep(3)

#책 제목 가져오기
soup = BeautifulSoup(driver.page_source, "lxml" )
watcha = []
titless = soup.select('li > a')

dataTitles = []
dataRating = []
for title in titless:
    reTitle = title.text
    rrrTitle = re.sub("★", ",", reTitle)
    rereTitle = re.split(',', rrrTitle)
    
    dataTitles.append(rereTitle[0])
    dataRating.append(rereTitle[-1])
        


df01 = pd.DataFrame(dataTitles)
df02 = pd.DataFrame(dataRating)
df03 = pd.concat([df01, df02], axis=1)
df03 = df03.drop(df03.index[0:3])
df03 = df03.drop(df03.index[-3:])
df03 = df03.reset_index(drop=True)
df03.to_csv('watcha10.csv', index = False, header=None, encoding = 'utf-8')